In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("./..")
from PARC.model import model_burgers as model
import PARC.data.data_burgers as data
from PARC import misc

c:\Users\phong\.conda\envs\tf\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.12.2 when it was built against 1.12.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
burgers_data = data.DataBurgers()
R_list = [1000, 2500, 5000, 7500, 10000]
a_list = [0.5, 0.6, 0.7, 0.8, 0.9]
w_list = [0.7, 0.8, 0.9, 1.0]
clipped_data = burgers_data.clip_raw_data('./data/burgers/train/' , R_list,a_list,w_list, purpose = 'training')

# Training


### Stage 1: Differentiator training

In [ ]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices(clipped_data[:,:,:,:3])
dataset_label = tf.data.Dataset.from_tensor_slices(clipped_data[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 10000) 
dataset = dataset.batch(8)

In [2]:
tf.keras.backend.clear_session()
parc = model.PARCv2_burgers(n_time_step = 1, step_size= 1/100, solver = "heun", mode = "differentiator_training")
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00005, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.differentiator.save_weights('PARC2_BURGER_diff_heun.h5')

### Stage 2: Data-driven integration training

In [ ]:
# Pretrain integrator
tf.keras.backend.clear_session()
parc = model.PARCv2_burgers(n_time_step = 2, step_size= 1/100, solver = "heun", mode = "integrator_training")
parc.differentiator.load_weights('./pretrained_weights/burgers/parc2_diff_burgers_heun.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.integrator.save_weights('PARC2_BURGER_int_heun.h5')

# Validation

In [3]:
R_list_test = [100, 500, 3000, 6500, 12500, 15000]
a_list_test = [0.35, 0.40, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.00]
w_list_test = [0.55, 0.6, 0.65, 0.75, 0.85, 0.95, 1.05]
clipped_data_test = burgers_data.clip_raw_data('./data/burgers/test/' , R_list_test, a_list_test, w_list_test, sequence_length = 100, purpose = 'validation')

## Load model from weight files

In [4]:
tf.keras.backend.clear_session()
parc = model.PARCv2_burgers(n_time_step = 100, step_size = 1/100, solver = "heun")

misc.geturl( url="https://virginia.box.com/shared/static/hpr7hol6cxt972dx2cvykgo6xqdwsv5k", title="PARC2_BURGER_diff_heun.h5" )
misc.geturl( url="https://virginia.box.com/shared/static/ohmg89i7klv3p8go8zumk4z114fhrg4f", title="PARC2_BURGER_int_heun.h5" )

parc.differentiator.load_weights('./pretrained_weights/burgers/parc2_diff_burgers_heun.h5')
parc.integrator.load_weights('./pretrained_weights/burgers/parc2_int_burgers_heun.h5')

## Make prediction on the test set

In [10]:
prediction_data = []
for j in range(2):
    input_seq_current = tf.cast(clipped_data_test[j:j+1,:,:,:3], dtype = tf.float32)
    res = parc.predict(input_seq_current)
    prediction_data.append(res)
    print('Finish case ', j)
prediction_data = np.concatenate(prediction_data, axis = 0)

Finish case  0
Finish case  1


In [ ]:
np.save('./results/burgers/parcv2_burgers.npy',prediction_data)

## Visualizing result

In [ ]:
# Load data
# # DNS
gt = np.load('../result/burgers/GT.npy')
# NeuralODE
parc = np.load('../result/burgers/neuralode_burgers.npy')
parc_data = np.load('../result/burgers/neuralode_burgers_data_int.npy')
# FNO
fno = np.load('../result/burgers/fno.npy')
# PhyCRNet
phycrnet = np.load('../result/burgers/phycrnet_burgers.npy')
# PI-FNO
pifno = np.load('../result/burgers/pifno.npy')
# PARCv2
parcv2 = np.load('../result/burgers/parcv2_burgers.npy')

y_pred = [parcv2, parc, parc_data, fno, phycrnet, pifno]

In [ ]:
# Plot field evolution
from parc.visualization import visualization_burgers as viz

viz.plot_field_evolution(y_pred, gt, case_id = 235)

In [ ]:
# Plot MSE, PDE residual graph
from parc import metrics

# Compute MSE and PDE residual
PdeLoss = metrics.BurgersPdeLoss(dt=2/100, dx=6/64)

# Defining auxillary functions
def rmse(pred,gt):
    return np.sqrt(np.mean((pred-gt)**2))

def get_loss_pinn(gt, pinn):
    idx = 0
    u_gt = np.sqrt(gt[:,:,:,0::3]**2+gt[:,:,:,1::3]**2)
    u_pinn = np.sqrt(pinn[:,:,:,0::3]**2+pinn[:,:,:,1::3]**2)
    mse_pinn = []
    pde_pinn = []
    for R in R_list:
        mse_pinn_a = []
        pde_pinn_a = []
        for a in a_list:
            mse_pinn_w = []
            pde_pinn_w = []
            for w in w_list:
                # Ground truth
                iter_gt = u_gt[idx:idx+1,:,:,:]
                # PhyCRNet
                iter_pinn = u_pinn[idx:idx+1,:,:,:]
                # MSE
                mse_pinn_w.append(rmse(iter_pinn,iter_gt))
                # PDE loss
                test = pinn[idx:idx+1]
                test_data = np.concatenate([test[:,:,:,0::3],test[:,:,:,1::3]])
                train_tensor = np.transpose(test_data.reshape(64, 64, 100, 2), (2, 3, 1, 0))
                PdeLoss.set_data(train_tensor)
                fu, fv = PdeLoss.ComputePdeLoss(1/R)
                pde_pinn_w.append((np.mean(np.sqrt(fu**2+fv**2))))
                idx += 1
                # print(idx)
            mse_pinn_a.append(mse_pinn_w)
            pde_pinn_a.append(pde_pinn_w)
        mse_pinn.append(mse_pinn_a)
        pde_pinn.append(pde_pinn_a)
    return np.array(mse_pinn), np.array(pde_pinn)

mse_whole = []
pde_whole = []

for i in range(6):
    mse_pinn, pde_pinn = get_loss_pinn(gt, y_pred[i])
    mse_whole.append(mse_pinn)
    pde_whole.append(pde_pinn)
    print(i)

In [ ]:
viz.pde_mse_R(mse_whole, pde_whole, R_list)